In [247]:
import os
from os import path
import glob
import pandas as pd
import json
from collections import defaultdict

In [248]:
# perf_dir = "/home/shtoshni/Research/events/models/perf"
perf_dir = "/share/data/speech/shtoshni/research/events/models/perf"
slurm_id_list = ["6425048"]  #"6407936"

files = []

for slurm_id in slurm_id_list:
    files.extend(sorted(glob.glob(path.join(perf_dir, slurm_id + "*")), 
                                  key=lambda x: int(path.splitext(path.basename(x))[0].split('_')[1])))
    
# files = sorted(files, key=lambda x: int(path.splitext(path.basename(x))[0].split('_')[1]))
print(len(files))

36


In [249]:
model_dict_list = []
for file in files:
    model_dict_list.append(json.loads(open(file).read()))

In [250]:
def determine_varying_attributes(model_dict_list, ignore_attribs=['model_dir', 'train', 'test', 'dev', 'pretrained_mention_model',
                                                                  'slurm_id', 'best_model_dir', 'data_dir']):
    attrib_to_vals = defaultdict(set)
    for model_dict in model_dict_list:
        for attrib, val in model_dict.items():
            if attrib in ignore_attribs:
                continue
            else:
                attrib_to_vals[attrib].add(val)
    
    
    varying_attribs = []
    for attrib, vals in attrib_to_vals.items():
        if len(vals) > 1:
            varying_attribs.append(attrib)
            
    return varying_attribs

In [251]:
varying_attribs = determine_varying_attributes(model_dict_list)


In [252]:
perf_df = pd.DataFrame(columns=(varying_attribs))

for model_dict in model_dict_list:
    perf_dict = {}
    for attrib in varying_attribs:
        perf_dict[attrib] = model_dict[attrib]
            
    perf_dict['fscore'] = round(model_dict['dev']['fscore'], 1)
    perf_dict['test_f'] = round(model_dict['test']['fscore'], 1)
                
    perf_df = perf_df.append(perf_dict, ignore_index=True)

In [253]:
idx = perf_df.groupby(['use_mem_context', 'event_subtype_loss_wt', 'top_span_ratio'])['fscore'].idxmax()
dev_max = perf_df.iloc[idx]
dev_max

,top_span_ratio,use_mem_context,event_subtype_loss_wt,seed,fscore,test_f
4,0.10,False,1.0,5,46.9,47.1
11,0.15,False,1.0,6,46.8,47.8
15,0.20,False,1.0,4,46.9,46.0
22,0.10,False,10.0,5,46.3,47.6
28,0.15,False,10.0,5,47.3,47.9
34,0.20,False,10.0,5,47.6,48.7
1,0.10,True,1.0,5,46.9,47.9
6,0.15,True,1.0,4,47.5,46.2
13,0.20,True,1.0,5,47.1,47.2
18,0.10,True,10.0,4,47.2,46.8


In [254]:
perf_df.groupby(['use_mem_context', 'event_subtype_loss_wt', 'top_span_ratio'])[['fscore', 'test_f']].mean()


fscore     test_f
use_mem_context event_subtype_loss_wt top_span_ratio                      
False           1.0                   0.10            46.333333  46.866667
                                      0.15            46.033333  47.600000
                                      0.20            46.433333  46.033333
                10.0                  0.10            46.033333  47.366667
                                      0.15            46.500000  47.633333
                                      0.20            46.566667  47.966667
True            1.0                   0.10            46.066667  46.733333
                                      0.15            46.666667  46.333333
                                      0.20            46.233333  46.833333
                10.0                  0.10            46.733333  46.733333
                                      0.15            46.466667  47.900000
                                      0.20            47.133333  47.033333

In [255]:
perf_df.groupby(['use_mem_context'])[['fscore', 'test_f']].mean()


,fscore,test_f
use_mem_context,,
False,46.316667,47.244444
True,46.550000,46.927778


In [256]:
perf_df.groupby(['event_subtype_loss_wt'])[['fscore', 'test_f']].mean()


,fscore,test_f
event_subtype_loss_wt,,
1.0,46.294444,46.733333
10.0,46.572222,47.438889


In [257]:
perf_df.groupby(['top_span_ratio'])[['fscore', 'test_f']].mean()


,fscore,test_f
top_span_ratio,,
0.10,46.291667,46.925000
0.15,46.416667,47.366667
0.20,46.591667,46.966667
